In [1]:
# import some basic libraries
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
from sklearn import linear_model

%matplotlib inline

In [2]:
# IMPORT BASIC DATA
EXPERIMENT_DATA = pickle.load(open('EXPERIMENT_SET_pandas.pkl', 'rb'))
EVALUATION_SET = pickle.load(open('EVALUATION_SET_pandas.pkl', 'rb'))
CONSIDERED = EXPERIMENT_DATA[EXPERIMENT_DATA["GRAD"] == "YES"]

sampleSize = 11932/258253
print(sampleSize)
print("Experiment_Data: {} \nEvaluation_Set: {} \
       \nGraduated_Set: {}".format(EXPERIMENT_DATA.shape, 
                                  EVALUATION_SET.shape,
                                  CONSIDERED.shape))
print("Experiment_Data Columns:\n{}".format(list(EXPERIMENT_DATA.columns.values)))
print("Evaluation_Set Columns:\n{}".format(list(EVALUATION_SET.columns.values)))

0.046202754663062964
Experiment_Data: (258253, 12) 
Evaluation_Set: (38, 4)        
Graduated_Set: (11932, 12)
Experiment_Data Columns:
['YEAR', 'EXPERIMENT', 'LOCATION', 'VARIETY', 'FAMILY', 'CHECK', 'RM', 'REPNO', 'YIELD', 'CLASS_OF', 'GRAD', 'BAGSOLD']
Evaluation_Set Columns:
['CLASS_OF', 'VARIETY', 'FAMILY', 'RM']


In [83]:
clean = EXPERIMENT_DATA[EXPERIMENT_DATA["GRAD"] != '.']
print(clean.shape)
clean.is_copy = False
# remove bags sold for now, we just want to predict graduation 
# del clean["BAGSOLD"]
# del clean["YEAR"]
print(clean.shape[0])

(21894, 12)
21894


In [84]:
var_dict = list(clean["VARIETY"].to_dict().values())
loc_dict = list(clean["LOCATION"].to_dict().values())
exp_dict = list(clean["EXPERIMENT"].to_dict().values())
family_dict = list(clean["FAMILY"].to_dict().values())

var_dict_eval = list(EVALUATION_SET["VARIETY"].to_dict().values())
family_dict_eval = list(EVALUATION_SET["FAMILY"].to_dict().values())

In [85]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le1 = preprocessing.LabelEncoder()
le2 = preprocessing.LabelEncoder()
le3 = preprocessing.LabelEncoder()
le4 = preprocessing.LabelEncoder()
le5 = preprocessing.LabelEncoder()

labeled_var = np.asarray(le.fit_transform(var_dict))
labeled_loc = np.asarray(le1.fit_transform(loc_dict))
labeled_exp = np.asarray(le2.fit_transform(exp_dict))
labeled_fam = np.asarray(le3.fit_transform(family_dict))

labeled_var_eval = np.asarray(le4.fit_transform(var_dict_eval))
labeled_fam_eval = np.asarray(le5.fit_transform(family_dict_eval))



In [86]:
grad_idx = clean.columns.get_loc("GRAD")

clean["GRAD"] = clean["GRAD"].replace(["YES", "NO"], [1,0])
clean["CHECK"] = clean["CHECK"].astype(int)
clean["VARIETY"] = labeled_var
clean["LOCATION"] = labeled_loc
clean["EXPERIMENT"] = labeled_exp
clean["FAMILY"] = labeled_fam
clean["CLASS_OF"] = clean["CLASS_OF"].astype(int)
clean["BAGSOLD"] = clean["BAGSOLD"].astype(int)

EVALUATION_SET["VARIETY"] = labeled_var_eval
EVALUATION_SET["FAMILY"] = labeled_fam_eval
# clean.to_csv("clean.csv")

In [87]:
print(clean.dtypes)
clean.head()

YEAR            int64
EXPERIMENT      int64
LOCATION        int64
VARIETY         int64
FAMILY          int64
CHECK           int32
RM            float64
REPNO           int64
YIELD         float64
CLASS_OF        int32
GRAD            int64
BAGSOLD         int32
dtype: object


,YEAR,EXPERIMENT,LOCATION,VARIETY,FAMILY,CHECK,RM,REPNO,YIELD,CLASS_OF,GRAD,BAGSOLD
694,2009,95,103,15,24,0,3.7,1,68.133720,2011,0,0
695,2009,95,103,15,24,0,3.7,2,74.836534,2011,0,0
1263,2009,95,103,19,1,0,3.8,1,48.027659,2011,0,0
1264,2009,95,103,19,1,0,3.8,2,52.497143,2011,0,0
1379,2009,198,48,74,29,0,2.1,1,61.924294,2011,1,61161


In [92]:
# partition the clean data for 60/40 train test
samples = int(clean.shape[0] * 0.6)
features = clean.columns.values[:-1]
print(features)
train_x = clean[0:samples][features]
del train_x["EXPERIMENT"]
del train_x["LOCATION"]
del train_x["CHECK"]
del train_x["REPNO"]
del train_x["YIELD"]
del train_x["YEAR"]
del train_x["GRAD"]
train_y = clean[0:samples]["GRAD"]
valid_x = clean[samples:][features]
del valid_x["EXPERIMENT"]
del valid_x["LOCATION"]
del valid_x["CHECK"]
del valid_x["REPNO"]
del valid_x["YIELD"]
del valid_x["YEAR"]
del valid_x["GRAD"]
valid_y = clean[samples:]["GRAD"]

test_x = EVALUATION_SET
print(EVALUATION_SET.dtypes)

print(train_x.shape)
print(train_x.columns.values)

['YEAR' 'EXPERIMENT' 'LOCATION' 'VARIETY' 'FAMILY' 'CHECK' 'RM' 'REPNO'
 'YIELD' 'CLASS_OF' 'GRAD']
CLASS_OF      int64
VARIETY       int64
FAMILY        int64
RM          float64
dtype: object
(13136, 4)
['VARIETY' 'FAMILY' 'RM' 'CLASS_OF']


In [93]:
from sklearn.naive_bayes import BernoulliNB

print(train_x.columns.values)
gnb = BernoulliNB()
gnb.fit(train_x, train_y)
print(valid_x.shape, test_x.shape)
valid_x.columns.values

['VARIETY' 'FAMILY' 'RM' 'CLASS_OF']
(8758, 4) (38, 4)


array(['VARIETY', 'FAMILY', 'RM', 'CLASS_OF'], dtype=object)

In [94]:
predictions = gnb.predict(valid_x)
print("Accuracy of the Model", np.sum(predictions == valid_y) / len(predictions))
new_pred = gnb.predict(test_x)
valid_x.shape
print(new_pred)

Accuracy of the Model 0.381708152546
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]
